In [48]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint
import pandas as pd
import numpy as np
import csv

In [53]:
pip install boto3


Note: you may need to restart the kernel to use updated packages.


In [54]:
def load_json(bank_name):
    mydict={"absa":'ABSA.json',
           "fnb":"FNB.json",
            "bidvest":"Bidvest.json",
            "standardbank":"Standard Bank.json",
            "nedbank":"Nedbank.json"
           }
    with open(mydict[bank_name]) as json_file: 
        data = json.load(json_file)
    
    return data[0]['Blocks']

In [55]:
def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text

In [56]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows

In [57]:
def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv


In [58]:
def generate_table_blockscode(bank_name):
    blocks=load_json(bank_name)
    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
         "<b> NO Table FOUND </b>"
            

    return table_blocks,blocks_map

In [59]:
def create_csv(bank_name):  
    '''
    creates a csv and given a bank name ;bank name is the name of the bank beng used by thr company 
    '''
    csv = ''
    table_blocks,blocks_map=generate_table_blockscode(bank_name)
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv.strip("\n")

In [144]:
csv=create_csv("absa")

In [149]:
csv

'Table: Table_1\n\nDate ,Transaction Description ,Charge ,,Debit Amount ,Credit Amount ,Balance ,\n5/03/2019 5/03/2019 ,Bal Brought Forward Airtime Debit Tiq001absa Card No. 4971 Mtn: 0730403965 ,1.50 ,T ,499.00 ,,4 794.48 4 295.48 ,\n7/03/2019 ,Acb Debit:External Settlement lairdressihairdressing V148 ,19.00 ,T ,3 662.00 ,,633.48 ,\n12/03/2019 ,Acb Credit Settlement Hollard Etimage004-0016 ,,,,6 298.80 ,6 932.28 ,\n15/03/2019 ,Acb Debit:Externa Settlement Centrafin Debit 033424:029008 ,19.00 ,T ,1 325.16 ,,5 607.12 ,\n18/03/2019 ,Airtime Debit Tiq001absa ,1.50 ,T ,110.00 ,,5 497.12 ,\n19/03/2019 ,Card No. 4971 Vodacom: 0824941097 bank Payment To Settlement Absa Bank Wire It Upstairs ,,,1 000.00 ,,4 497.12 ,\n27/03/2019 ,Npf Credit Treas/lbs ,,,,10 000.00 ,14 497.12 ,\n27/03/2019 ,Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Settlement ,,,13 000.00 ,,1 497.12 ,\n29/03/2019 ,1400154585 Absa Bank 1400154585 Acb Debit:External Settlement ,19.00 ,T ,591.82 ,,905.30 ,\n,Nashua Ec 7127 Nashua

In [151]:
def generate_info(csv,bank_name='Absa'):
    
    if bank_name.lower()=='absa': 
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
          
        
    elif bank_name.lower()=='fnb':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.rfind(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_3\n\n","") 
        
    elif bank_name.lower()=='bidvest':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
  
    elif bank_name.lower()=='nedbank':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
          
    elif bank_name.lower()=='standardbank':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
        
    
    return bankstatement,banksummary
        
    
    
  




In [152]:
def writecsv(csv,output_table1,output_table2):

    """" Write bank statement and bank summary csv """
    bankstatement_tuple=generate_info(csv)
    with open(output_table1, "wt") as bank:
        bank.write(str(bankstatement_tuple[0]))
    with open(output_table2, "wt") as bank:
        bank.write(str(bankstatement_tuple[1]))
    
        

In [64]:
writecsv(csv,"bank_statement.csv","bank_summary.csv")

In [154]:
writecsv(fnbcsv,"bank_statement.csv","bank_summary.csv")

In [65]:
bank_df=pd.read_csv("bank_statement.csv")

In [160]:
fnbbank_df=pd.read_csv("bank_statement.csv")

In [161]:
summary_df=pd.read_csv("bank_summary.csv").reset_index()

ParserError: Error tokenizing data. C error: Expected 4 fields in line 12, saw 6


In [157]:
fnbsummary_df=pd.read_csv("bank_summary.csv").reset_index()

ParserError: Error tokenizing data. C error: Expected 4 fields in line 12, saw 6


In [67]:
print(summary_df)

                      index  Account Summary:  Unnamed: 1 Unnamed: 2  \
0  Balance Brought Forward                 NaN      4 794        48    
1           Sundry Credits                 NaN     51 298        80    
2            Sundry Debits                 NaN     52 416       70-    
3                  Charges                 NaN      1 920       50-    
4                  Balance                 NaN      1 756        08    
5          Overdraft Limit                 NaN          0        00    

   Unnamed: 3  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  


In [68]:
print(summary_df.columns)

Index(['index', 'Account Summary: ', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')


In [69]:
summary_df=summary_df[['index', 'Unnamed: 1', 'Unnamed: 2',]]  

In [70]:
summary_df.columns=['Account Summary', 'Rands', 'Cents']

In [71]:
summary_df

Account Summary   Rands Cents
0  Balance Brought Forward    4 794   48 
1           Sundry Credits   51 298   80 
2            Sundry Debits   52 416  70- 
3                  Charges    1 920  50- 
4                  Balance    1 756   08 
5          Overdraft Limit        0   00

In [72]:
amount=lambda x,y:x+'.'+y

In [73]:
summary_df["Amount"]=amount(summary_df["Rands"],summary_df["Cents"])

In [74]:
summary_df

Account Summary   Rands Cents       Amount
0  Balance Brought Forward    4 794   48     4 794.48 
1           Sundry Credits   51 298   80    51 298.80 
2            Sundry Debits   52 416  70-   52 416.70- 
3                  Charges    1 920  50-    1 920.50- 
4                  Balance    1 756   08     1 756.08 
5          Overdraft Limit        0   00         0.00

In [75]:
summary_df.drop(["Rands","Cents"],axis=1,inplace=True)

In [76]:
summary_df

Account Summary       Amount
0  Balance Brought Forward     4 794.48 
1           Sundry Credits    51 298.80 
2            Sundry Debits   52 416.70- 
3                  Charges    1 920.50- 
4                  Balance     1 756.08 
5          Overdraft Limit         0.00

In [77]:
Sign= lambda X:'-' if X.strip().endswith('-') else "+"

In [78]:
summary_df["sign"]=summary_df["Amount"].apply(Sign)

In [79]:
import ast

In [80]:
summary_df["Amount"]=summary_df["Amount"].apply(lambda X:float(X.replace(" ","").replace("-","")))

In [81]:
summary_df.T

0                1               2  \
Account Summary  Balance Brought Forward   Sundry Credits   Sundry Debits    
Amount                            4794.48          51298.8         52416.7   
sign                                    +                +               -   

                        3         4                 5  
Account Summary  Charges   Balance   Overdraft Limit   
Amount             1920.5   1756.08                 0  
sign                    -         +                 +

In [82]:
amount_df = summary_df.T.loc[["Amount"]]

In [83]:
balance = amount_df[4].values[0]
bbf = amount_df[0].values[0]

In [84]:
balance

1756.08

In [85]:
bbf

4794.48

In [86]:
bank_df.head()

Date                            Transaction Description   \
0  5/03/2019 5/03/2019   Bal Brought Forward Airtime Debit Tiq001absa C...   
1            7/03/2019   Acb Debit:External Settlement lairdressihairdr...   
2           12/03/2019      Acb Credit Settlement Hollard Etimage004-0016    
3           15/03/2019   Acb Debit:Externa Settlement Centrafin Debit 0...   
4           18/03/2019                           Airtime Debit Tiq001absa    

   Charge  Unnamed: 3 Debit Amount  Credit Amount             Balance   \
0      1.5         T        499.00             NaN  4 794.48 4 295.48    
1     19.0         T      3 662.00             NaN             633.48    
2      NaN        NaN           NaN      6 298.80            6 932.28    
3     19.0         T      1 325.16             NaN           5 607.12    
4      1.5         T        110.00             NaN           5 497.12    

   Unnamed: 7  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN

In [87]:
bank_df.tail()

Date           Transaction Description   Charge  Unnamed: 3  \
13          NaN  1400154585 Absa Bank 1400154585       NaN        NaN   
14  30/03/2019      Internet Bank Fee Settlement       NaN        *k    
15   1/04/2019        Monthly Acc Fee Headoffice       NaN        *k    
16   1/04/2019     Transaction Charge Headoffice       NaN        *k    
17          NaN                               NaN      NaN        NaN   

   Debit Amount  Credit Amount    Balance   Unnamed: 7  
13           NaN            NaN        NaN         NaN  
14     1 052.50             NaN  9 852.80          NaN  
15        75.00             NaN  9 777.80          NaN  
16       193.00             NaN  9 584.80          NaN  
17           NaN            NaN        NaN         NaN

In [88]:
summary_df.head().T

0                1               2  \
Account Summary  Balance Brought Forward   Sundry Credits   Sundry Debits    
Amount                            4794.48          51298.8         52416.7   
sign                                    +                +               -   

                        3         4  
Account Summary  Charges   Balance   
Amount             1920.5   1756.08  
sign                    -         +

In [89]:
df_statement=pd.read_json('ABSA.json')

In [90]:
df_statement['Blocks']

0    [{'BlockType': 'PAGE', 'Geometry': {'BoundingB...
1    [{'BlockType': 'WORD', 'Confidence': 99.950119...
Name: Blocks, dtype: object

In [91]:
output_file = 'output.csv'

# replace content
with open(output_file, "wt")as fout:
      fout.write(str(csv))

      fout.close()



In [92]:
statement_df= pd.read_csv('bank_statement.csv')

In [93]:
statement_df.replace({np.NaN:0},inplace=True)

In [94]:
statement_df

Date                            Transaction Description   \
0   5/03/2019 5/03/2019   Bal Brought Forward Airtime Debit Tiq001absa C...   
1             7/03/2019   Acb Debit:External Settlement lairdressihairdr...   
2            12/03/2019      Acb Credit Settlement Hollard Etimage004-0016    
3            15/03/2019   Acb Debit:Externa Settlement Centrafin Debit 0...   
4            18/03/2019                           Airtime Debit Tiq001absa    
5            19/03/2019   Card No. 4971 Vodacom: 0824941097 bank Payment...   
6            27/03/2019                               Npf Credit Treas/lbs    
7            27/03/2019   Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Se...   
8            29/03/2019   1400154585 Absa Bank 1400154585 Acb Debit:Exte...   
9                      0                          Nashua Ec 7127 Nashua Ec    
10           30/03/2019                               Npf Credit Treas/lbs    
11                     0                     Eftbb45jjd3pf002/Images 1 Fnb    
12           30/03/2019                           bank Transfer Settlement    
13                     0                   1400154585 Absa Bank 1400154585    
14           30/03/2019                       Internet Bank Fee Settlement    
15            1/04/2019                         Monthly Acc Fee Headoffice    
16            1/04/2019                      Transaction Charge Headoffice    
17                     0                                                  0   

    Charge  Unnamed: 3 Debit Amount  Credit Amount             Balance   \
0       1.5         T        499.00               0  4 794.48 4 295.48    
1      19.0         T      3 662.00               0             633.48    
2       0.0          0             0      6 298.80            6 932.28    
3      19.0         T      1 325.16               0           5 607.12    
4       1.5         T        110.00               0           5 497.12    
5       0.0          0     1 000.00               0           4 497.12    
6       0.0          0             0     10 000.00           14 497.12    
7       0.0          0    13 000.00               0           1 497.12    
8      19.0         T        591.82               0             905.30    
9       0.0          0             0              0                   0   
10      0.0          0             0     15 000.00           15 905.30    
11      0.0          0             0              0                   0   
12      0.0          0     5 000.00               0           0 905.30    
13      0.0          0             0              0                   0   
14      0.0        *k      1 052.50               0           9 852.80    
15      0.0        *k         75.00               0           9 777.80    
16      0.0        *k        193.00               0           9 584.80    
17      0.0          0             0              0                   0   

    Unnamed: 7  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0  
5          0.0  
6          0.0  
7          0.0  
8          0.0  
9          0.0  
10         0.0  
11         0.0  
12         0.0  
13         0.0  
14         0.0  
15         0.0  
16         0.0  
17         0.0

In [95]:
statement_df=statement_df[['Date ', 'Transaction Description ', 'Charge ',
       'Debit Amount ', 'Credit Amount ', 'Balance ']]

In [96]:
statement_df=statement_df[statement_df['Date ']!= 0]

In [97]:
statement_df

Date                            Transaction Description   \
0   5/03/2019 5/03/2019   Bal Brought Forward Airtime Debit Tiq001absa C...   
1             7/03/2019   Acb Debit:External Settlement lairdressihairdr...   
2            12/03/2019      Acb Credit Settlement Hollard Etimage004-0016    
3            15/03/2019   Acb Debit:Externa Settlement Centrafin Debit 0...   
4            18/03/2019                           Airtime Debit Tiq001absa    
5            19/03/2019   Card No. 4971 Vodacom: 0824941097 bank Payment...   
6            27/03/2019                               Npf Credit Treas/lbs    
7            27/03/2019   Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Se...   
8            29/03/2019   1400154585 Absa Bank 1400154585 Acb Debit:Exte...   
10           30/03/2019                               Npf Credit Treas/lbs    
12           30/03/2019                           bank Transfer Settlement    
14           30/03/2019                       Internet Bank Fee Settlement    
15            1/04/2019                         Monthly Acc Fee Headoffice    
16            1/04/2019                      Transaction Charge Headoffice    

    Charge  Debit Amount  Credit Amount             Balance   
0       1.5       499.00               0  4 794.48 4 295.48   
1      19.0     3 662.00               0             633.48   
2       0.0             0      6 298.80            6 932.28   
3      19.0     1 325.16               0           5 607.12   
4       1.5       110.00               0           5 497.12   
5       0.0     1 000.00               0           4 497.12   
6       0.0             0     10 000.00           14 497.12   
7       0.0    13 000.00               0           1 497.12   
8      19.0       591.82               0             905.30   
10      0.0             0     15 000.00           15 905.30   
12      0.0     5 000.00               0           0 905.30   
14      0.0     1 052.50               0           9 852.80   
15      0.0        75.00               0           9 777.80   
16      0.0       193.00               0           9 584.80

In [98]:
statement_df.iloc[0]

Date                                                     5/03/2019 5/03/2019 
Transaction Description     Bal Brought Forward Airtime Debit Tiq001absa C...
Charge                                                                    1.5
Debit Amount                                                          499.00 
Credit Amount                                                               0
Balance                                                    4 794.48 4 295.48 
Name: 0, dtype: object

In [99]:
statement_df.iloc[0][["Date "]] =  "5/03/2019"

<ipython-input-99-24a64be7027f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statement_df.iloc[0][["Date "]] =  "5/03/2019"
/Users/tamalaameliamanda/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1098: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_labels(key, value)


In [100]:
statement_df.iloc[0, statement_df.columns.get_loc('Date ')] = "5/03/2019"

In [101]:
statement_df.iloc[0, statement_df.columns.get_loc('Balance ')] = "4 295.48"

In [102]:
statement_df.iloc[0, statement_df.columns.get_loc('Transaction Description ')] = "Airtime Debit Tiq001absa Card No. 4971 Mtn: 0730403965"

In [103]:
statement_df["Debit Amount "]=statement_df["Debit Amount "].astype(object).apply(lambda x: str(x).replace(" ", '')).astype(float)

# Convert to string, astype object, remove spaces and convert back to float 

In [121]:
statement_df["Credit Amount "]=statement_df["Credit Amount "].astype(object).apply(lambda x: str(x).replace(" ", '')).astype(float)

In [122]:
statement_df

Date                            Transaction Description   Charge   \
0     5/03/2019  Airtime Debit Tiq001absa Card No. 4971 Mtn: 07...      1.5   
1    7/03/2019   Acb Debit:External Settlement lairdressihairdr...     19.0   
2   12/03/2019      Acb Credit Settlement Hollard Etimage004-0016       0.0   
3   15/03/2019   Acb Debit:Externa Settlement Centrafin Debit 0...     19.0   
4   18/03/2019                           Airtime Debit Tiq001absa       1.5   
5   19/03/2019   Card No. 4971 Vodacom: 0824941097 bank Payment...      0.0   
6   27/03/2019                               Npf Credit Treas/lbs       0.0   
7   27/03/2019   Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Se...      0.0   
8   29/03/2019   1400154585 Absa Bank 1400154585 Acb Debit:Exte...     19.0   
10  30/03/2019                               Npf Credit Treas/lbs       0.0   
12  30/03/2019                           bank Transfer Settlement       0.0   
14  30/03/2019                       Internet Bank Fee Settlement       0.0   
15   1/04/2019                         Monthly Acc Fee Headoffice       0.0   
16   1/04/2019                      Transaction Charge Headoffice       0.0   

    Debit Amount   Credit Amount     Balance   
0          499.00             0.0    4 295.48  
1         3662.00             0.0     633.48   
2            0.00          6298.8   6 932.28   
3         1325.16             0.0   5 607.12   
4          110.00             0.0   5 497.12   
5         1000.00             0.0   4 497.12   
6            0.00         10000.0  14 497.12   
7        13000.00             0.0   1 497.12   
8          591.82             0.0     905.30   
10           0.00         15000.0  15 905.30   
12        5000.00             0.0   0 905.30   
14        1052.50             0.0   9 852.80   
15          75.00             0.0   9 777.80   
16         193.00             0.0   9 584.80

In [123]:
#Liquidity Function

def generate_liquidity(balance_brought_forward,balance):
    if balance_brought_forward > balance:
        return "low"
    elif balance_brought_forward == balance:
        return "medium"
    else: 
        return "High"


In [124]:
liquidity= generate_liquidity(bbf,balance)

In [125]:
print(liquidity)

low


In [126]:
statement_df

Date                            Transaction Description   Charge   \
0     5/03/2019  Airtime Debit Tiq001absa Card No. 4971 Mtn: 07...      1.5   
1    7/03/2019   Acb Debit:External Settlement lairdressihairdr...     19.0   
2   12/03/2019      Acb Credit Settlement Hollard Etimage004-0016       0.0   
3   15/03/2019   Acb Debit:Externa Settlement Centrafin Debit 0...     19.0   
4   18/03/2019                           Airtime Debit Tiq001absa       1.5   
5   19/03/2019   Card No. 4971 Vodacom: 0824941097 bank Payment...      0.0   
6   27/03/2019                               Npf Credit Treas/lbs       0.0   
7   27/03/2019   Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Se...      0.0   
8   29/03/2019   1400154585 Absa Bank 1400154585 Acb Debit:Exte...     19.0   
10  30/03/2019                               Npf Credit Treas/lbs       0.0   
12  30/03/2019                           bank Transfer Settlement       0.0   
14  30/03/2019                       Internet Bank Fee Settlement       0.0   
15   1/04/2019                         Monthly Acc Fee Headoffice       0.0   
16   1/04/2019                      Transaction Charge Headoffice       0.0   

    Debit Amount   Credit Amount     Balance   
0          499.00             0.0    4 295.48  
1         3662.00             0.0     633.48   
2            0.00          6298.8   6 932.28   
3         1325.16             0.0   5 607.12   
4          110.00             0.0   5 497.12   
5         1000.00             0.0   4 497.12   
6            0.00         10000.0  14 497.12   
7        13000.00             0.0   1 497.12   
8          591.82             0.0     905.30   
10           0.00         15000.0  15 905.30   
12        5000.00             0.0   0 905.30   
14        1052.50             0.0   9 852.80   
15          75.00             0.0   9 777.80   
16         193.00             0.0   9 584.80

In [127]:
statement_df["Debit Amount "].max()

13000.0

In [129]:
statement_df["Debit Amount "].sum()

26508.48

In [128]:
statement_df['Credit Amount '].max()

15000.0

In [130]:
statement_df['Credit Amount '].sum()

31298.8

In [131]:
cashinflow=statement_df['Credit Amount '].sum()

In [132]:
cashinflow

31298.8

In [133]:
cashoutflow=statement_df["Debit Amount "].sum()

In [134]:
cashoutflow

26508.48